In [6]:
import sqlite3
import pandas as pd

from data.setup_db import add_dataset_to_db

In [13]:

conn = sqlite3.connect('/Users/maxwellpatterson/Desktop/personal/bball/bball-reports/data/player_data.db')

df = pd.read_sql_query("SELECT * FROM play_by_play", conn)

game_info_df = pd.read_sql_query("SELECT game_id, game_date FROM game_info", conn)

# Convert game_date to datetime and extract year
game_info_df['year'] = pd.to_datetime(game_info_df['game_date']).dt.year


# Merge play-by-play data with game_info
df = pd.merge(df, game_info_df[['game_id', 'year']], on='game_id', how='left')
print(df.to_string())

: 

In [ ]:
# Create a function to extract player actions
def extract_player_action(row):
    for desc in ['homedescription', 'neutraldescription', 'visitordescription']:
        if pd.notna(row[desc]):
            return row[desc]
    return None

In [9]:
# Add a new column for player actions
df['player_action'] = df.apply(extract_player_action, axis=1)

# Create a list of player columns
player_columns = ['player1_name', 'player2_name', 'player3_name']

# Melt the dataframe to create rows for each player involved in an action
melted_df = pd.melt(df, id_vars=['game_id', 'year', 'eventnum', 'player_action'], 
                    value_vars=player_columns, 
                    var_name='player_number', value_name='player_name')

# Remove rows where player_name is None
melted_df = melted_df.dropna(subset=['player_name'])

# Sort the dataframe
melted_df = melted_df.sort_values(['game_id', 'eventnum', 'player_number'])

# Display the first few rows of the new dataframe
print(melted_df.head(50).to_string())

conn.close()

     game_id  year  eventnum                                player_action player_number       player_name
1   29600012  1996         2  Jump Ball O'Neal vs. Kleine: Tip to Cassell  player1_name  Shaquille O'Neal
26  29600012  1996         2  Jump Ball O'Neal vs. Kleine: Tip to Cassell  player2_name        Joe Kleine
51  29600012  1996         2  Jump Ball O'Neal vs. Kleine: Tip to Cassell  player3_name       Sam Cassell
2   29600012  1996         3                   MISS Cassell 15' Jump Shot  player1_name       Sam Cassell
3   29600012  1996         4                 O'Neal REBOUND (Off:0 Def:1)  player1_name  Shaquille O'Neal
4   29600012  1996         5              MISS Ceballos 26' 3PT Jump Shot  player1_name   Cedric Ceballos
5   29600012  1996         6                Cassell REBOUND (Off:0 Def:1)  player1_name       Sam Cassell
6   29600012  1996         7                      Van Exel P.FOUL (P1.T1)  player1_name     Nick Van Exel
7   29600012  1996         8            Cassel

In [12]:
df.shape

(25, 36)

In [11]:
len(melted_df['player_action'].unique())

23